In [13]:
import xarray as xr
import pandas as pd
import pickle as pk
import re
import requests
import os
import gc

In [22]:
# Sensor E: url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T104012-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181020T213838-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# Sensor F url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T041943-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
# url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/jazlynnatalie12@gmail.com/20190712T124024454Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/RS03ECAL/MJ03E/06-BOTPTA302/streamed/botpt_nano_sample'

tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]

In [23]:
# print(datasets[1])
print(datasets)

[]


In [21]:
# ds = xr.open_dataset(datasets[1])
ds = xr.open_dataset(datasets())

TypeError: 'list' object is not callable

In [10]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/2019FifteenSecond_mean_dataE/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [11]:
# read in the data directly off THREDDS and write out as subsampled pickled pandas dataframe
# NOTE: It takes about one hour to subsample 69499.81 Mbytes of data and write it out to a dataframe. 
num = 0
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('.25T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = '/home/jovyan/data/botpt/2019FifteenSecond_mean_dataE/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
    num = num +1

    with open(out, 'wb') as fh:
        pk.dump(pressure_min,fh)

    gc.collect()

In [12]:
# create a single file with all the pickled data.
pressure_min = pd.DataFrame()
for path, subdirs, files in os.walk('/home/jovyan/data/botpt/2019FifteenSecond_mean_dataE/'):
    for name in files:
        file_name = os.path.join(path, name) 
        with open(file_name, 'rb') as f:
            pd_df = pk.load(f)
            pressure_min = pressure_min.append(pd_df)

with open('/home/jovyan/data/botpt/2019bottom_pressure15s_E.pkl', 'wb') as f:
    pk.dump(pressure_min,f)